In [1]:
import os
ON_KAGGLE = os.path.exists('/kaggle/input')
ENABLE_SUBMISSION = False
os.environ["ON_KAGGLE"] = str(ON_KAGGLE)

In [2]:
# Install required packages if on Kaggle
if ON_KAGGLE:
    !pip install -U /kaggle/input/accelerate-wheelwhl/accelerate-0.29.1-py3-none-any.whl -qq
    !pip install -U /kaggle/input/bitsandbytes-0-42-0-py3-none-any-whl/bitsandbytes-0.42.0-py3-none-any.whl -qq
    !pip install -U /kaggle/input/hydracore120py3/hydra_core-1.2.0-py3-none-any.whl -qq
    !pip install -U /kaggle/input/omegaconf222py3/omegaconf-2.2.2-py3-none-any.whl -qq
    !pip install -U /kaggle/input/parglare-0-18-0/parglare-0.18.0-py3-none-any.whl -qq
    !pip install -U /kaggle/input/rank-bm25-whl/rank_bm25-0.2.2-py3-none-any.whl -qq
    !pip install -U /kaggle/input/peft-0-11-1/peft-0.11.1-py3-none-any.whl -qq
    !pip install -U /kaggle/input/trl-0-9-4/trl-0.9.4-py3-none-any.whl -qq
    !pip install -U /kaggle/input/amino-gaz-whl/aimo_gaz-0.0.1-py3-none-any.whl -qq --no-deps --force-reinstall
else:
    !pip install -r requirements.txt
    !pip install --upgrade build
    !python -m build
    !pip install dist/aimo_gaz-0.0.1-py3-none-any.whl --force-reinstall

In [3]:
from aimo_gaz.models.model import Model

2024-06-17 03:08:47.358606: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-17 03:08:47.358705: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-17 03:08:47.524963: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [4]:
# Model from Hugging Face hub
# Test model
import os
import numpy as np
import json
model_name = "Salesforce/codet5-small"
is_seq2seq = "llama-2" not in model_name.lower()
token = None
model = Model(model_name, token=token, use_lora=False, is_seq2seq=is_seq2seq)
main_prompt = "Do simple math problems (Answer only the number and use '[END]' to finish the response):\nQuestion: 2 + 2\nAnswer: 4\n[END]"
with model:
    for response in model.generate(
            [
                f"{main_prompt}\nQuestion: 4 + 5\nAnswer:",
                f"{main_prompt}\nQuestion: 2 + 2\nAnswer:",
                f"{main_prompt}\nQuestion: 3 + 3 * 3\nAnswer:",
            ],
            max_new_tokens=10,
            temperature=0.1, # Nucleus sampling
            do_sample=True, # Nucleus sampling
            top_k=5, # Nucleus sampling
            # num_beams=5, # Beam search
            num_return_sequences=5,
            stop_tokens=["[END]"],#, model._tokenizer.eos_token],
            skip_special_tokens=True,
            padding=True,
            #truncation=True,
            compute_probabilities=True,
            return_full_text=False):

        print("-" * 50)
        print(f"Prompt: \n{response.input_text}")
        for idx, result in enumerate(response.generated_text):
            probability = np.exp(-response.neg_log_likelihood[idx])
            print(f"Result [{idx + 1}] ({probability*100}% chance): {result}")
        print("-" * 50)

tokenizer_config.json:   0%|          | 0.00/1.48k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/703k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/294k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/12.5k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.57k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/242M [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


--------------------------------------------------
Prompt: 
Do simple math problems (Answer only the number and use '[END]' to finish the response):
Question: 2 + 2
Answer: 4
[END]
Question: 4 + 5
Answer:
Result [1] (12.23933346818143% chance): +
Result [2] (12.23933346818143% chance): +
Result [3] (12.23933346818143% chance): +
Result [4] (12.23933346818143% chance): +
Result [5] (12.23933346818143% chance): +
--------------------------------------------------
--------------------------------------------------
Prompt: 
Do simple math problems (Answer only the number and use '[END]' to finish the response):
Question: 2 + 2
Answer: 4
[END]
Question: 2 + 2
Answer:
Result [1] (12.24612865450534% chance): 2
Result [2] (10.630733594873238% chance): + 2
Answer:
Result [3] (11.293638149041161% chance): 4
[END]
Result [4] (10.630733594873238% chance): + 2
Answer:
Result [5] (11.481983390617492% chance): +
--------------------------------------------------
--------------------------------------

In [ ]:
# Set up the evaluation API
if ENABLE_SUBMISSION and ON_KAGGLE:
    import aimo

    env = aimo.make_env()
    iter_test = env.iter_test()

    # Iterate through the test set and use the model make predictions
    for test, sample_submission in iter_test:
        sample_submission['answer'] = 100 # TODO: call the model to predict the solution for text['problem']
        env.predict(sample_submission)
        print(test)
        print(sample_submission, '\n')